In [5]:
fname='test'
seed=0

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb
import logging
from tqdm import tqdm_notebook
import itertools

import pickle as pkl

import time

start = time.time()
chunks = 5000000
remain_df = None

for i_c, df in tqdm_notebook(enumerate(pd.read_csv('../input/test_set.csv', 
                                     chunksize=chunks, 
                                     iterator=True)),
                             total=91):
    # Check object_ids
    # I believe np.unique keeps the order of group_ids as they appear in the file
    unique_ids = np.unique(df['object_id'])
    new_remain_df = df.loc[df['object_id'] == unique_ids[-1]].copy()

    if remain_df is None:
        df = df.loc[df['object_id'].isin(unique_ids[:-1])].copy()
    else:
        df = pd.concat([remain_df, df.loc[df['object_id'].isin(unique_ids[:-1])]], axis=0)

    # Create remaining samples df
    remain_df = new_remain_df

    with open('../input/test_chunk_%d.csv' %i_c, 'wb') as file:
        pkl.dump(df, file)
        
with open('../input/test_chunk_%d.csv' %100, 'wb') as file:
    pkl.dump(remain_df, file)
